In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [8]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here is the latest news summary on Amazon:

1. Amazon is investing up to $50 billion to expand its AI and supercomputing infrastructure, aimed at supporting U.S. government agencies.
2. Recent AI innovations include enhanced visual product search (Lens Live), advanced AI models for AWS customers, and generative AI applications such as Video Recaps for Prime Video customers.
3. Amazon is expanding international infrastructure with a subsea cable connecting Maryland and County Cork to deliver faster and more reliable cloud and AI services.
4. Prime members benefit from various perks like discounted fuel savings at thousands of gas stations and fast same-day deliveries from micro fulfillment centers, such as in the UAE.
5. Amazon offers comprehensive employee benefits including health care from day one, mental health support, and education programs like Career Choice to help employees develop new skills.
6. The company highlighted leadership insights from veteran employees and its workforce investment in adapting to economic changes.

If you want, I can also find specific market or stock-related news or deeper details on any of these topics.

### Look at the trace

https://platform.openai.com/traces

In [9]:
deepu_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("deepu").reset(deepu_initial_strategy)

display(Markdown(await read_accounts_resource("deepu")))
display(Markdown(await read_strategy_resource("deepu")))

{"name": "deepu", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-12-02 09:37:25", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [10]:
agent_name = "deepu"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is deepu and your account is under your name, deepu.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "deepu", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-12-02 09:37:25", 10000.0], ["2025-12-02 09:37:28", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so fa

### And to run our Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions Taken

1. **Market Research**:
   - Investigated the current market trends in the technology and energy sectors.
   - Positive sentiment in technology, especially in big players like Apple (AAPL) and Microsoft (MSFT).
   - Energy sector benefiting from higher oil prices and geopolitical tensions.

2. **Stock Purchases**:
   - **Apple (AAPL)**: Bought **30 shares** at approximately $283.67. Rationale: Strong player in the tech sector with recent gains.
   - **Microsoft (MSFT)**: Attempted to buy **20 shares**, but insufficient funds.
   - **ExxonMobil (XOM)**: Attempted to buy **25 shares**, but insufficient funds.
   - **Chevron (CVX)**: Attempted to buy **20 shares**, but insufficient funds.

### Updated Portfolio:
- **Current Holdings**:
  - **AAPL**: 30 shares
- **Cash Balance**: $1,490.01
- **Portfolio Value**: $9,983.01, showing a slight loss.

### Next Steps:
- With limited funds remaining, I can focus on selling a portion of existing holdings if opportunities arise or monitoring the market for potential trades that align with the day trading strategy. If you want to adjust any holdings or explore new opportunities, please let me know!

### Then go and look at the trace

http://platform.openai.com/traces


In [13]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "deepu", "balance": 1490.0139999999992, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 30}, "transactions": [{"symbol": "AAPL", "quantity": 30, "price": 283.6662, "timestamp": "2025-12-02 09:38:01", "rationale": "The tech sector is showing cautious optimism with recent gains, and Apple (AAPL) is a strong player in the consumer electronics area."}], "portfolio_value_time_series": [["2025-12-02 09:37:25", 10000.0], ["2025-12-02 09:37:28", 10000.0], ["2025-12-02 09:38:01", 9983.014], ["2025-12-02 09:38:19", 9983.014]], "total_portfolio_value": 9983.014, "total_profit_loss": -16.986000000000786}'

In [14]:
from traders import Trader


In [15]:
trader = Trader("deepu")

In [16]:
await trader.run()

In [17]:
await read_accounts_resource("deepu")

'{"name": "deepu", "balance": 24.61905999999931, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 30, "ON": 29}, "transactions": [{"symbol": "AAPL", "quantity": 30, "price": 283.6662, "timestamp": "2025-12-02 09:38:01", "rationale": "The tech sector is showing cautious optimism with recent gains, and Apple (AAPL) is a strong player in the consumer electronics area."}, {"symbol": "ON", "quantity": 29, "price": 50.53086, "timestamp": "2025-12-02 09:39:08", "rationale": "Strong earnings growth and potential in the semiconductor market, supported by buyback plans."}], "portfolio_value_time_series": [["2025-12-02 09:37:25", 10000.0], ["2025-12-02 09:37:28", 10000.0], ["2025-12-02 09:38:01", 9983.014], ["2025-12-02 09:38:19", 9983.014], ["2025-12-02 09:38:27", 9983.014], ["2025-12-02 09:39:08", 9980.089059999998], ["2025-12-02 09:39:14", 9980.089059999998]], "total_portfolio_value": 9980.089059999998, "total

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [18]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
